# Adding AgentCore Memory

In this lab, you will add Amazon Bedrock Agentcore Memory as the conversation manager for the Strands Agent, which will give both short term memory for the agent, as well as RAG based personalization for users of the agent.

We will demonstrate how Strands will use the Memory with just a small code change and then how to configure the Long Term memory for personalization.

## Prerequisites
- Complete Lab 3.1 first.
- AgentCore runtime permissions configured

### Understanding AgentCore Memory

AgentCore Memory provides two types of memory for your agents:

**Short-Term Memory (Conversation Continuity)**
- Maintains context within a single conversation session
- Remembers what was discussed earlier in the conversation
- Enables natural multi-turn interactions
- Implemented through **SummaryStrategy** that creates conversation summaries

**Long-Term Memory (User Preferences)**
- Learns user preferences across multiple sessions
- Stores personalization data using RAG-based retrieval
- Enables the agent to remember user-specific information over time
- Implemented through **UserPreferenceStrategy** that extracts and stores preferences

Without memory, each agent invocation is stateless - it has no knowledge of previous interactions. With memory, your agent can maintain context and provide personalized experiences.

Import required libraries and initialize the AgentCore Runtime client to manage memory-enabled agent deployment:

In [ ]:
import json
import os
import random, string
import time

import boto3
from bedrock_agentcore_starter_toolkit import Runtime

boto_session = boto3.Session()
region = boto_session.region_name

print(f"current region: {region}")
account_id = boto_session.client("sts").get_caller_identity()["Account"]
print(f"current account: {account_id}")

agentcore_runtime = Runtime()

### Step 1: Create Memory Resource with Strategies

We'll create a memory resource with two strategies:

**SummaryStrategy:**
- Automatically summarizes conversation history
- Stores summaries in namespaces organized by actor (user) and session
- Helps the agent understand conversation context without processing entire history

**UserPreferenceStrategy:**
- Extracts and stores user preferences from conversations
- Organizes preferences by actor ID for personalization
- Enables the agent to remember user-specific information across sessions

The MemoryManager handles the lifecycle of these memory resources and their associated strategies.

In [ ]:
from bedrock_agentcore.memory.session import MemorySessionManager
from bedrock_agentcore_starter_toolkit.operations.memory.manager import MemoryManager
from bedrock_agentcore_starter_toolkit.operations.memory.models.strategies import SummaryStrategy, UserPreferenceStrategy

memory_manager = MemoryManager(region_name=region)

memory_response = memory_manager.get_or_create_memory(
    name="intelligent_rag_memory",
    strategies=[
        SummaryStrategy(
            name="SessionSummarizer",
            namespaces=[
                "/summaries/{actorId}/{sessionId}"
            ]
        ),
        UserPreferenceStrategy(
            name="UserPreferencesLearner",
            namespaces=["/users/{actorId}/preferences"]
        )
    ]
)

memory_id = memory_response.get('id')
print(f"Created memory with ID: {memory_id}")

### Step 2: Store Memory ID for Runtime Access

The memory ID needs to be accessible to the agent runtime when it's deployed to AgentCore. We store it in SSM Parameter Store because:

- **Runtime Access**: AgentCore can retrieve the ID at startup without hardcoding
- **Security**: SSM provides secure parameter storage with IAM-based access control
- **Flexibility**: Easy to update the memory ID without redeploying the agent
- **Consistency**: Same pattern used for knowledge base IDs

The agent code will retrieve this parameter when initializing the memory client.

In [ ]:
param_name = '/app/intelligent_rag/agentcore/memory_id'

ssm = boto3.client("ssm")
ssm.put_parameter(Name=param_name, Value=memory_id, Type="String", Overwrite=True)
print(f"Stored {memory_id} in SSM: {param_name}")

### Step 3: Understand the Memory Integration Approach

The `intelligent_rag_agent_runtime_with_memory.py` uses **AgentCoreMemorySessionManager** for native memory integration:

**Approach:**
- Uses `AgentCoreMemorySessionManager` from Strands Agents SDK
- Automatic conversation persistence
- Built-in retrieval from memory namespaces
- Configurable retrieval per namespace (top_k, relevance_score)

**Key Components:**
1. **AgentCoreMemoryConfig**: Configures memory ID, session ID, actor ID, and retrieval settings
2. **RetrievalConfig**: Defines how to retrieve from each memory namespace (summaries, preferences)
3. **AgentCoreMemorySessionManager**: Handles automatic memory persistence and retrieval

**Memory Namespaces:**
- `/summaries/{actorId}/{sessionId}`: Short-term conversation summaries
- `/users/{actorId}/preferences`: Long-term user preferences

Get the notebook directory path to locate the memory-enabled agent code and updated IAM policy:

In [ ]:
import os
from pathlib import Path

import ipynbname

try:
    # Get the notebook name and path
    notebook_path = ipynbname.path()
    notebook_dir = Path(notebook_path.parent)
except:
    notebook_dir = Path.resolve()

print(f"notebook_dir: {notebook_dir}")

# Agentcore starter toolkit expects the files in the current directory
# So changing our working directory.
os.chdir(notebook_dir)
print(f"changed working directory to: {notebook_dir}")

Load the IAM policy with memory permissions and create/update the execution role for the memory-enabled agent:

In [ ]:
# Load policy from external file
with open('policy.json', 'r') as f:
    policy_template = f.read()

# load trust policy from external file
with open('trust-policy.json', 'r') as f:
    trust_policy_template = f.read()

policy = policy_template.replace('REGION', region).replace('ACCOUNT_ID', account_id).replace('REPO_ARN', '*')
print(f"Policy loaded and updated with region: {region}, account: {account_id}")

trust_policy = trust_policy_template.replace('REGION', region).replace('ACCOUNT_ID', account_id)

# create IAM role using the policies
suffix = random.choices(string.ascii_lowercase + string.digits, k=8)
iam_client = boto3.client('iam')
role_name = f"bedrock-runtime-execution-role-{''.join(suffix)}"

role = iam_client.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument=trust_policy
)

iam_client.put_role_policy(
    RoleName=role_name,
    PolicyName='bedrock-runtime-execution-policy',
    PolicyDocument=policy
)

### Step 4: Deploy Updated Agent with Memory

The memory-enabled agent uses a different entrypoint file: `intelligent_rag_agent_runtime_with_memory.py`

**Key Differences:**
- Initializes MemorySessionManager for conversation management
- Registers MemoryHookProvider to capture conversation events
- Retrieves memory ID from SSM Parameter Store
- Configures actor and session IDs for proper memory organization

The deployment process is identical to the base agent, but the runtime behavior now includes automatic memory management.

Trigger the deployment of the memory-enabled agent to AgentCore Runtime:

In [ ]:
agent_name = "intelligent_rag_agent_with_memory"
response = agentcore_runtime.configure(
    entrypoint="intelligent_rag_agent_runtime_with_memory.py",
    execution_role=role['Role']['Arn'],
    #auto_create_execution_role=True,   # If you don't pass in an execution role, AgentCore can create a minimal execution role for you.
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
    # memory_mode="STM_AND_LTM"   # NOTE:  We created the memory above. You can also have AgentCore Toolkit create the memory for you.
)
response

Monitor deployment status and store the agent ARN for use in subsequent notebooks:

In [ ]:
launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
print(status)

# Store the agent ARN for use in other notebooks
if status == 'READY':
    try:
        agent_arn = status_response.endpoint['agentRuntimeArn']
        %store agent_arn
        print(f"Agent ARN stored for use in other notebooks: {agent_arn}")
    except Exception as e:
        print(f"Could not store agent ARN: {e}")

### Step 5: Test Conversation Continuity

When invoking the memory-enabled agent, the **session_id** parameter is crucial:

**Same Session ID:**
- Agent maintains conversation context
- Can reference previous messages
- Builds on earlier discussion

**Different Session ID:**
- Starts a fresh conversation
- No access to previous session context
- Useful for testing or new conversation threads

**Actor ID:**
- Identifies the user across sessions
- Enables long-term preference learning
- Allows personalization based on user history

**Note on Query Performance:** Queries may take 2+ minutes to complete as the agent makes 10+ tool calls to route between knowledge bases, retrieve data, and generate responses. Optimizing agentic systems often involves reducing the number of LLM generation calls and the size of LLM outputs. As you add tools and features, you may need to review agent trajectories and optimize performance. In Lab 3.3, you'll learn how to use AgentCore Observability to analyze these patterns and identify optimization opportunities.


Create a convenience function that includes session tracking to enable memory persistence across invocations:

In [ ]:
import json

def print_response_text(invoke_response):
    response = invoke_response['response']
    
    # If it's a list, join all parts first
    if isinstance(response, list):
        response = ''.join(response)
    
    # Parse JSON
    response = json.loads(response)
    
    # Extract the text content
    text = response['result']['content'][0]['text']
    print(text)

In [ ]:
def ask(question, session):
    print(f"Query: {question}")
    print("-" * 100)
    response = agentcore_runtime.invoke(
        payload={"prompt": question},
        session_id=session)
    print_response_text(response)
    print("\n")

In [ ]:
ask(
    question="How many customers reviewed product_890, are those reviews positive or negative?", 
    session="session111111111111111111111111111")

In [ ]:
# Try explicitly stopping the session and asking a question about previous conversations

# To stop a session:
agentcore_runtime.stop_session(session_id="session111111111111111111111111111")

In [ ]:
ask(
    question="What review did I just lookup?", 
    session="session111111111111111111111111111")

### Test Memory Continuity and User Preferences

The agent now has memory capabilities for conversation continuity and user preferences. Let's test these features:"""

* Try asking the same question with a different session ID
* Try asking the same session ID about previous questions

In [ ]:
#  ask questions with a different session ID
#ask(
#    question="How many customers purhcased product_890?",
#    session="session222222222222222222222222222222")
#
#ask(
#    question="What review did I just lookup?",
#    session="session111111111111111111111111111")
#
# # To stop a session:
# agentcore_runtime.stop_session(session_id="xxxxx")

## Next Steps

Your agent now has memory capabilities for conversation continuity and user preferences!

**Ready to continue?** Proceed to [**Lab 3.3**](3.3-agentcore-observability.ipynb) to explore AgentCore Observability features including CloudWatch dashboards, traces, and transaction search.
